## Practicas de la creacion de la DB de libros

In [4]:
import sqlite3

conn = sqlite3.connect('books_data.db')

'''
Un cursor es una estructura de datos que permite recorrer los resultados de una consulta SQL, fila por fila, en lugar de recibir todos los resultados a la ve
'''

cursor = conn.cursor()

################################## BLOQUE 1 - Creacion de tablas ##################################
# Tabla para los autores
cursor.execute(
    '''CREATE TABLE IF NOT EXISTS autores 
    (id INTEGER PRIMARY KEY AUTOINCREMENT, nombre_completo TEXT)'''
)
#conn.commit()

# Tabla para los libros
cursor.execute(
    '''CREATE TABLE IF NOT EXISTS libros 
    (id INTEGER PRIMARY KEY AUTOINCREMENT, 
    titulo TEXT, 
    precio REAL, 
    disponibilidad TEXT, 
    link TEXT, 
    rating TEXT, 
    descripcion LONGTEXT, 
    categoria TEXT, 
    UPC TEXT)'''
)
#conn.commit()

# Tabla relacional libros-autores
cursor.execute(
    '''CREATE TABLE IF NOT EXISTS libros_autores 
    (libro_id INTEGER, autor_id INTEGER,
    PRIMARY KEY (libro_id, autor_id),
    FOREIGN KEY (libro_id) REFERENCES libros(id),
    FOREIGN KEY (autor_id) REFERENCES autores(id))'''
)
conn.commit()

################################## BLOQUE 2 - Leer el JSON con los datos ##################################
import json 

def guardar_json(archivo_json, lista_diccionarios):
    try:
        with open(archivo_json, 'r', encoding='utf-8') as contenido:
            datos = json.load(contenido)

            # Se checkea si el JSON ya es una lista de diccionarios
            if isinstance(datos, list):     # isinstance(object, classinfo) sirve para checkear si un objeto es una instancia de una clase/subclase especifica. retorna True o False
                # se asigna directamente a la lista
                lista_diccionarios.extend(datos)        # extend es un metodo de las listas para agregar elementos de una estructura de datos al final de una lista existente
            # Si el json es solo un diccionario
            elif isinstance(datos, dict):
                # se agrega a la lista
                lista_diccionarios.append(datos)
            else:
                print('El archivo JSON no contiene una lista ni diccionario validos')

        return lista_diccionarios
    except FileNotFoundError:
        print(f'El archivo json {archivo_json} no existe')

################################## BLOQUE 3 - Insertar datos en las tablas ##################################
'''
El segundo parámetro de cursor.execute() espera una tupla o secuencia de valores que se correspondan con los signos ? en la consulta SQL.
'''

def insertar_datos(informacion_libros):
    # recorrer cada libro dentro del archivo
    for libro in informacion_libros:
        # campos que vamos a utilizar del dict (titulo, precio, disponibilidad, link, rating, descripcion, categoria, UPC)
        titulo = libro["Titulo"]
        ##precio = libro["Precio"]
        disponibilidad = libro["Disponibilidad"]
        link = libro["Link"]
        rating = libro["Rating"]
        descripcion = libro["Descripcion"]
        categoria = libro["Categoria"]
        upc = libro["UPC"]

        # limpieza del campo precio (ya que el campo precio esta definido como REAL(float))
        precio_limpio = float(libro["Precio"].replace("Â£", "").strip())

        # insertar sus campos
        cursor.execute(
            '''INSERT INTO libros (titulo, precio, disponibilidad, link, rating, descripcion, categoria, UPC) 
            VALUES (?,?,?,?,?,?,?,?)''', (titulo, precio_limpio, disponibilidad, link, rating, descripcion, categoria, upc))
        
        # Guardar el id de cada libro insertado
        libro_id = cursor.lastrowid

        for autor in libro['Autores']:
            # hacemos una consulta con select para saber si ya existe un autor con su respectivo id dentro de la tabla (EVITAR DUPLICADOS)
            # Significa Select 'id' from 'table' where 'column_name' equals 'some_value'
            cursor.execute('''SELECT id FROM autores WHERE nombre_completo = ?''', (autor,))
            resultado = cursor.fetchone()       # fetch se utiliza para recuperar filas de datos de un cursor
            if resultado:   # si ya existe ese autor
                autor_id = resultado[0] # se deja ese id encontrado
            else:   # si no existe, se inserta al autor
                cursor.execute('''INSERT INTO autores (nombre_completo) VALUES (?)''', (autor,))
                autor_id = cursor.lastrowid # y guardamos su id

            # Poblar libros_autores con los id respectivos
            cursor.execute('''INSERT INTO libros_autores (libro_id, autor_id) VALUES (?,?)''', (libro_id, autor_id))
        
    conn.commit()

################################## Lllamado a las funciones ##################################
libros_info = []
guardar_json('autores_googlebooks.json', libros_info)
insertar_datos(libros_info)

cursor.close()
conn.close()